In [1]:
import os

# 원본 라벨 파일 경로 및 YOLO 포맷 라벨 저장 경로
input_label_dir = (
    "C:/Users/user/Desktop/aws_s3_Data/labels"  # 원본 라벨 파일이 있는 폴더
)
output_label_dir = (
    "C:/Users/user/Desktop/aws_s3_Data/labels_format"  # YOLO 포맷으로 저장할 폴더
)

# 출력 폴더 생성
os.makedirs(output_label_dir, exist_ok=True)


# YOLO 포맷으로 변환하는 함수
def convert_to_yolo_format(label_file, output_dir):
    with open(label_file, "r", encoding="utf-8") as f:
        lines = f.readlines()

    bboxs_section = False
    yolo_labels = []

    # bboxs 섹션만 처리
    for line in lines:
        if "[bboxs]" in line:
            bboxs_section = True
            continue
        if "[question_text]" in line:
            bboxs_section = False
            break

        if bboxs_section and line.strip():  # bboxs 섹션 내부
            values = line.strip().split()
            if len(values) == 5:  # YOLO 포맷: class, x_center, y_center, width, height
                class_id = int(values[0])  # 클래스 ID (0: 텍스트, 1: 비텍스트)
                x_center = float(values[1])
                y_center = float(values[2])
                width = float(values[3])
                height = float(values[4])
                yolo_labels.append(f"{class_id} {x_center} {y_center} {width} {height}")

    # YOLO 포맷 파일 저장
    output_file = os.path.join(output_dir, os.path.basename(label_file))
    output_file = output_file.replace(".txt", ".txt")  # 동일한 이름으로 저장
    with open(output_file, "w", encoding="utf-8") as f:
        f.write("\n".join(yolo_labels))


# 디렉토리 내 모든 파일 변환
for label_file in os.listdir(input_label_dir):
    if label_file.endswith(".txt"):
        convert_to_yolo_format(
            os.path.join(input_label_dir, label_file), output_label_dir
        )

print(f"YOLO 포맷 라벨 파일이 {output_label_dir}에 저장되었습니다.")

YOLO 포맷 라벨 파일이 C:/Users/user/Desktop/aws_s3_Data/labels_format에 저장되었습니다.


In [2]:
import os
import shutil
import random

# 데이터 경로
DATASET_ROOT = "C:/Users/user/Desktop/aws_s3_Data"
IMAGE_DIR = os.path.join(DATASET_ROOT, "images")
LABEL_DIR = os.path.join(DATASET_ROOT, "labels_format")

# 분리된 데이터 저장 경로
TRAIN_IMAGE_DIR = os.path.join(DATASET_ROOT, "images/train")
VAL_IMAGE_DIR = os.path.join(DATASET_ROOT, "images/val")
TRAIN_LABEL_DIR = os.path.join(DATASET_ROOT, "labels/train")
VAL_LABEL_DIR = os.path.join(DATASET_ROOT, "labels/val")

# 폴더 생성
os.makedirs(TRAIN_IMAGE_DIR, exist_ok=True)
os.makedirs(VAL_IMAGE_DIR, exist_ok=True)
os.makedirs(TRAIN_LABEL_DIR, exist_ok=True)
os.makedirs(VAL_LABEL_DIR, exist_ok=True)

# 이미지와 라벨 파일 리스트 가져오기
image_files = [f for f in os.listdir(IMAGE_DIR) if f.endswith((".jpg", ".png"))]
label_files = [f for f in os.listdir(LABEL_DIR) if f.endswith(".txt")]

# 데이터 검증: 이미지와 라벨 매칭 확인
image_files = sorted(image_files)
label_files = sorted(label_files)
assert len(image_files) == len(label_files), "이미지와 라벨 파일 수가 일치하지 않습니다."

# 데이터를 랜덤하게 섞기
data = list(zip(image_files, label_files))
random.seed(42)  # 재현성을 위해 고정
random.shuffle(data)

# 학습/검증 비율 설정
train_ratio = 0.8
train_count = int(len(data) * train_ratio)

train_data = data[:train_count]
val_data = data[train_count:]

# 파일 이동
def move_files(data, src_image_dir, src_label_dir, dest_image_dir, dest_label_dir):
    for image_file, label_file in data:
        shutil.move(os.path.join(src_image_dir, image_file), os.path.join(dest_image_dir, image_file))
        shutil.move(os.path.join(src_label_dir, label_file), os.path.join(dest_label_dir, label_file))

# 학습 데이터 이동
move_files(train_data, IMAGE_DIR, LABEL_DIR, TRAIN_IMAGE_DIR, TRAIN_LABEL_DIR)
# 검증 데이터 이동
move_files(val_data, IMAGE_DIR, LABEL_DIR, VAL_IMAGE_DIR, VAL_LABEL_DIR)

print(f"학습 데이터 {len(train_data)}개, 검증 데이터 {len(val_data)}개로 분리되었습니다.")


학습 데이터 12178개, 검증 데이터 3045개로 분리되었습니다.


In [3]:
from ultralytics import YOLO
import yaml
import os

# 1. 모델 초기화
model = YOLO("yolov8n.pt")  # Pretrained YOLOv8 모델 로드

# 2. 데이터 경로 설정 및 검증
DATASET_ROOT = "C:/Users/user/Desktop/aws_s3_Data"
data_config = {
    "path": DATASET_ROOT,  # 데이터셋 루트 디렉토리
    "train": "images/train",  # 학습 이미지 경로
    "val": "images/val",  # 검증 이미지 경로
    "names": ["text", "non-text"],  # 클래스 이름
}

data_yaml_path = os.path.join(DATASET_ROOT, "data.yaml")

# 3. 데이터 설정 파일 저장 (data.yaml 생성)
with open(data_yaml_path, "w") as file:
    yaml.dump(data_config, file)

# 4. YOLOv8 학습
print("==== YOLOv8 학습 시작 ====")
model.train(
    data=data_yaml_path,  # 데이터셋 설정 파일
    epochs=20,  # 학습 에포크 수 (10~20 추천)
    imgsz=640,  # 이미지 크기 (기본 640, 큰 객체는 512로 축소 가능)
    batch=8,  # CPU에서 학습 시 작은 배치 크기 (GPU 사용 시 16~32 추천)
    device="cpu",  # GPU 사용 시 '0' 또는 'cuda'
    name="text_nontext_classifier",  # 결과 저장 디렉토리 이름
    verbose=True,  # 학습 로그를 상세히 출력
    patience=3,  # EarlyStopping: 성능 개선 없으면 3 에포크 후 종료
    optimizer="Adam",  # Adam 최적화 알고리즘 사용
    workers=4,  # 데이터 로더 스레드 수 (CPU에 따라 조정)
    augment=True,  # 데이터 증강 활성화
)

# 5. 모델 검증
print("==== YOLOv8 검증 시작 ====")
results = model.val(
    data=data_yaml_path,  # 검증에 사용할 데이터셋 설정 파일
    imgsz=640,  # 검증 이미지 크기
    batch=8,  # CPU에서 학습 시 작은 배치 크기 (GPU 사용 시 조정 가능)
    device="cpu",  # GPU 또는 CPU 사용
)

# 검증 결과 출력
print("==== 검증 결과 ====")
print(f"Precision: {results.box.map50:.4f}")
print(f"Recall: {results.box.map:.4f}")
print(f"mAP@50: {results.box.map50:.4f}")
print(f"mAP@50-95: {results.box.map:.4f}")

# 6. 훈련된 모델 저장
model_path = os.path.join(DATASET_ROOT, "yolov8_text_nontext.pt")
model.save(model_path)  # 최종 모델 저장
print(f"최종 훈련된 모델이 저장되었습니다: {model_path}")

==== YOLOv8 학습 시작 ====
New https://pypi.org/project/ultralytics/8.3.54 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.48  Python-3.9.20 torch-2.5.1+cpu CPU (12th Gen Intel Core(TM) i7-12700)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=C:/Users/user/Desktop/aws_s3_Data\data.yaml, epochs=20, time=None, patience=3, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=cpu, workers=4, project=None, name=text_nontext_classifier5, exist_ok=False, pretrained=True, optimizer=Adam, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=No

train: Scanning C:\Users\user\Desktop\aws_s3_Data\labels\train... 12178 images, 0 backgrounds, 0 corrupt: 100%|██████████| 12178/12178 [00:11<00:00, 1038.77it/s]

train: WARNING  C:\Users\user\Desktop\aws_s3_Data\images\train\P3_1_03_34290_161336.png: 1 duplicate labels removed
train: WARNING  C:\Users\user\Desktop\aws_s3_Data\images\train\P4_1_05_23128_63810.png: 1 duplicate labels removed
train: WARNING  C:\Users\user\Desktop\aws_s3_Data\images\train\P4_1_06_23352_65124.png: 1 duplicate labels removed
train: WARNING  C:\Users\user\Desktop\aws_s3_Data\images\train\P4_2_04_40324_157628.png: 1 duplicate labels removed
train: WARNING  C:\Users\user\Desktop\aws_s3_Data\images\train\P4_2_06_33364_129538.png: 1 duplicate labels removed
train: WARNING  C:\Users\user\Desktop\aws_s3_Data\images\train\P4_2_06_33366_129748.png: 1 duplicate labels removed
train: WARNING  C:\Users\user\Desktop\aws_s3_Data\images\train\P5_1_01_34423_137097.png: 1 duplicate labels removed
train: WARNING  C:\Users\user\Desktop\aws_s3_Data\images\train\P5_1_03_42328_164871.png: 1 duplicate labels removed
train: WARNING  C:\Users\user\Desktop\aws_s3_Data\images\train\P5_1_06_426

train: New cache created: C:\Users\user\Desktop\aws_s3_Data\labels\train.cache


val: Scanning C:\Users\user\Desktop\aws_s3_Data\labels\val... 3045 images, 0 backgrounds, 0 corrupt: 100%|██████████| 3045/3045 [00:02<00:00, 1017.68it/s]

...

                   all       3045      10078      0.978      0.977      0.987      0.948
                  text       3044       9066      0.987      0.984      0.994      0.963
              non-text       1006       1012       0.97       0.97       0.98      0.934
Speed: 0.4ms preprocess, 44.7ms inference, 0.0ms loss, 0.2ms postprocess per image
Results saved to c:\Users\user\runs\detect\text_nontext_classifier5
==== YOLOv8 검증 시작 ====
Ultralytics 8.3.48  Python-3.9.20 torch-2.5.1+cpu CPU (12th Gen Intel Core(TM) i7-12700)
Model summary (fused): 168 layers, 3,006,038 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning C:\Users\user\Desktop\aws_s3_Data\labels\val.cache... 3045 images, 0 backgrounds, 0 corrupt: 100%|██████████| 3045/3045 [00:00<?, ?it/s]

val: WARNING  C:\Users\user\Desktop\aws_s3_Data\images\val\P5_2_05_40640_158112.png: 1 duplicate labels removed
val: WARNING  C:\Users\user\Desktop\aws_s3_Data\images\val\P6_2_05_40958_159702.png: 1 duplicate labels removed



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 381/381 [02:25<00:00,  2.62it/s]


                   all       3045      10078      0.978      0.978      0.987      0.947
                  text       3044       9066      0.986      0.985      0.994      0.963
              non-text       1006       1012      0.971      0.971       0.98      0.932
Speed: 0.4ms preprocess, 44.2ms inference, 0.0ms loss, 0.2ms postprocess per image
Results saved to c:\Users\user\runs\detect\text_nontext_classifier52
==== 검증 결과 ====
Precision: 0.9868
Recall: 0.9472
mAP@50: 0.9868
mAP@50-95: 0.9472
최종 훈련된 모델이 저장되었습니다: C:/Users/user/Desktop/aws_s3_Data\yolov8_text_nontext.pt
